In [1]:
###### Comment Legend ######

    # Describing a function
    ### Describing an issue
    ###### Infomational ######

###### To Do List ######

    # Create a function to save bad images for later use

In [1]:
import cv2
import os
import math
import mmcv
import torch
import pandas as pd
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
from ultralytics import YOLO
from ultralytics.yolo.utils.plotting import Annotator

from mmpose.apis import inference_top_down_pose_model, init_pose_model, vis_pose_result


# %run utils/segmentation/detect.ipynb
%run utils/superimposition/functions.ipynb
%run utils/extraction/objClass.ipynb
%run utils/skeleton/detect_skeletons.ipynb

C:\Users\gmald\anaconda3\envs\senior\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
class Configuration:
    def __init__(self, minimalImg, mainImg, segModel, BBModel, videoFormat, FPS, fileName, skip):
        self.minimalImg = minimalImg
        self.mainImg = mainImg
        
        segPath = "utils/segmentation/"
        
        self.segWeights = segPath+segModel
        self.BBWeights = segPath+BBModel
        
        self.fourcc = cv2.VideoWriter_fourcc(*videoFormat)
        
        self.FPS = FPS
        
        self.mainPath = f"videos/{fileName}/"
        self.processedPath = self.mainPath+"processed/"
        
        self.skip = skip
        
        self.pose_config = "utils/skeleton/demo/hrnet_w32_coco_256x192.py"
        self.pose_checkpoint = "https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth"
        
fileList = os.listdir("videos")
fileName = fileList[2]

print(fileList, "\nSelected File:", fileName)

config = Configuration(minimalImg="Thermal", mainImg="RGB", segModel="yolov7-seg.pt", BBModel="yolov7-tiny.pt", videoFormat="mp4v", FPS=10, fileName=fileName, skip=190 )

['2022-12-01_18-03-33', '2022-12-02_13-05-04', '2023-01-24_15-28-00', '2023-01-28_15-50-17', 'combined_outisde_13.gif', 'newStereo', 'outside_test', 'SII'] 
Selected File: 2023-01-24_15-28-00


In [4]:
###### Video Processor ######
%run utils/superimposition/functions.ipynb

# Load the models
seg_model = YOLO("yolov8n-seg.pt")

# BB_model = loadModel(config.BBWeights)
pose_model = init_pose_model(config.pose_config, config.pose_checkpoint, device)

# Grab videos from final location selected
rgbVideo = cv2.VideoCapture(f"{config.mainPath}rgbout.mp4")
thermalVideo = cv2.VideoCapture(f"{config.mainPath}thermalout.mp4")
depthVideo = cv2.VideoCapture(f"{config.mainPath}stereoout.mp4")
leftVideo = cv2.VideoCapture(f"{config.mainPath}leftout.mp4")

# Frame counter
calibrationCount = 0
frameCount = 0

rgbVideo.set(cv2.CAP_PROP_POS_FRAMES, config.skip)
thermalVideo.set(cv2.CAP_PROP_POS_FRAMES, config.skip)
depthVideo.set(cv2.CAP_PROP_POS_FRAMES, config.skip)
leftVideo.set(cv2.CAP_PROP_POS_FRAMES, config.skip)

# ObjList = Object_List()

TenScales = [ [1, 1, 1] ]
TenOffsets = [ [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]] ]

images = {"RGB": None, "Thermal": None, "Left": None, "Depth": None}

# Actual Loop which goes through all frames and moves as appropriate
while(True): 

    # Read from all videos a single frame
    ret, images["RGB"] = rgbVideo.read()
    ret1, images["Thermal"] = thermalVideo.read()
    ret2, images["Depth"] = depthVideo.read()
    ret3, images["Left"] = leftVideo.read()
    if not (ret & ret1 & ret2 and ret3):
        break
        
    
    # Every 100th frame preform calibration 
    if calibrationCount % 2 == 0:

        scales, offset = calibration(images.copy(), seg_model, config.minimalImg)
        
        if scales != None and offset != None:
            if TenScales[0] == [1,1,1]:
                del TenScales[0]
                del TenOffsets[0]
            
            TenScales.append(scales)
            TenOffsets.append(offset)

        if len(TenScales) > 30:
            radDel = int(np.random.rand()*30)
            
            del TenScales[radDel]
            del TenOffsets[radDel]
        
        scales, offset = filter_offsets_scales(TenScales, TenOffsets)
        scales = {"RGB": scales[0], "Thermal": scales[1], "Depth": scales[2]}
        offset = {"RGB": offset[0], "Thermal": offset[1], "Depth": offset[2]}

    # Convert stereoframe into colors for visualization 
    images["Depth"] = cv2.cvtColor(images["Depth"], cv2.COLOR_BGR2GRAY)
    
    # Modify images based on calibration settings
    images["Unrelated"] = superimpose_images(images.copy(), addColor=True)
    images = modify_images(images.copy(), scales, offset, images[config.minimalImg].shape[0])
    
    images["Related"] = superimpose_images(images.copy(), addColor=True)
    
    StereoframeMapped = cv2.normalize(images["Depth"], None, 255, 0, cv2.NORM_INF, cv2.CV_8UC1)
    StereoframeMapped = cv2.equalizeHist(StereoframeMapped)
    StereoframeMapped = cv2.applyColorMap(StereoframeMapped, cv2.COLORMAP_JET)
    images["Color_Depth"] = StereoframeMapped
    
    # Produce superimposed Image based on previous relationships
    # images["Superimposed"] = apply_mask(images.copy(), segmentation_model, scales, yoloResults[0].masks.data.cpu(), config.mainImg)

    # Create video to process 
    if calibrationCount==0:

        thermalout = cv2.VideoWriter(f"{config.processedPath}Thermal_sii.mp4", config.fourcc, config.FPS, (images[config.minimalImg].shape[:2][::-1]))
        stereoout = cv2.VideoWriter(f"{config.processedPath}Depth_sii.mp4", config.fourcc, config.FPS, (images[config.minimalImg].shape[:2][::-1]), 0)
        rgbout = cv2.VideoWriter(f"{config.processedPath}RGB_sii.mp4", config.fourcc, config.FPS, (images[config.minimalImg].shape[:2][::-1]))
        related_SIIout = cv2.VideoWriter(f"{config.processedPath}Related_SII.mp4", config.fourcc, config.FPS, (images[config.minimalImg].shape[:2][::-1]))
        unrelated_SIIout = cv2.VideoWriter(f"{config.processedPath}Unrelated_SII.mp4", config.fourcc, config.FPS, (images[config.minimalImg].shape[:2][::-1]))

    images["Related"] = cv2.resize(images["Related"], images[config.minimalImg].shape[:2][::-1])
    images["Unrelated"] = cv2.resize(images["Unrelated"], images[config.minimalImg].shape[:2][::-1])
    
    
    for label in images:
        cv2.imshow(label, images[label])
    
    # Write frames to all the videos
    thermalout.write(images["Thermal"])
    stereoout.write(images["Depth"])
    rgbout.write(images["RGB"])
    related_SIIout.write(images["Related"])
    unrelated_SIIout.write(images["Unrelated"])

    # Increment frame count
    calibrationCount+=1
    frameCount+=1
    
    # isd = input()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
print("\nProcessing Ended. Saving videos...")
        
    
# Save all videos
rgbout.release()
thermalout.release()
stereoout.release()
related_SIIout.release()
unrelated_SIIout.release()

print("Saving complete!")


### Create a function that grqabs all frames that broke the function

cv2.destroyAllWindows()

load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth

Processing Ended. Saving videos...
Saving complete!


In [12]:
rgbout.release()
thermalout.release()
stereoout.release()
related_SIIout.release()
unrelated_SIIout.release()

In [20]:
%run utils/superimposition/functions.ipynb
%run utils/extraction/objClass.ipynb

seg_model = YOLO("yolov8n-seg.pt")
pose_model = init_pose_model(config.pose_config, config.pose_checkpoint, device)

Handler = ObjectHandler()
images = {"RGB": None, "Thermal": None, "Depth": None}

PATH = f"{config.processedPath}{config.mainImg}_sii.mp4"

rgbVideo = cv2.VideoCapture(f"{config.processedPath}RGB_sii.mp4")
thermalVideo = cv2.VideoCapture(f"{config.processedPath}Thermal_sii.mp4")
depthVideo = cv2.VideoCapture(f"{config.processedPath}Depth_sii.mp4")

trackout = cv2.VideoWriter(f"{config.processedPath}track.mp4", config.fourcc, config.FPS, (512,512))

frameCount = 0
for result in seg_model.track(source=PATH, stream=True, verbose=False, classes=[0,2], device=0):
    
    ret, images["RGB"] = rgbVideo.read()
    ret1, images["Thermal"] = thermalVideo.read()
    ret2, images["Depth"] = depthVideo.read()
    if not (ret & ret1 & ret2):
        break
        
    
    StereoframeMapped = cv2.normalize(images["Depth"], None, 255, 0, cv2.NORM_INF, cv2.CV_32F)
    StereoframeMapped = cv2.convertScaleAbs(StereoframeMapped)
    StereoframeMapped = cv2.cvtColor(StereoframeMapped, cv2.COLOR_BGR2GRAY)
    StereoframeMapped = cv2.equalizeHist(StereoframeMapped)
    StereoframeMapped = cv2.applyColorMap(StereoframeMapped, cv2.COLORMAP_JET)
    images["Color_Depth"] = StereoframeMapped
    
    
    if result.boxes:
        
        # Collect Properties of Objects
        classes = result.boxes.cls
        masks = result.masks.data.cpu()
        boxes_array = np.array(result.boxes.xyxy.cpu())
        print(result.boxes)
        track_ids = result.boxes.id.cpu()
        frame_num = np.ones_like(classes) * frameCount
        object_results = [{'class': cls.item(),  'track_id': track_id.item(), 'bbox': bbox,  'mask': mask, 'frame_num': frame} 
                          for cls, track_id, bbox, mask, frame in zip(classes, track_ids, boxes_array, masks, frame_num)]
        
        images["Superimposed"] = apply_mask(images.copy(), masks, config.mainImg)
        
        object_results = inference_top_down_pose_model(pose_model, images[config.mainImg], object_results, format='xyxy')[0]
        
        Handler.appendObjects(images, object_results)
        images["Superimposed"] = vis_pose_result(pose_model, images["Superimposed"], object_results, bbox_color=(0,0,0))
    
        annotator = Annotator(images["Superimposed"])

        for obj in object_results:

            color = Handler.object_container[obj["track_id"]].color_id
            
            
            annotator.box_label(obj["bbox"], label=f'{obj["track_id"]}', color=color)
    
    frame = annotator.result()
    
    for label in images:
        cv2.imshow(label, images[label])
    
    cv2.imshow("YOLO", frame)
    
    trackout.write(images["Superimposed"])
    
    frameCount+=1
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

trackout.release()

load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth
tensor([[182.50389, 222.00867, 216.63762, 330.81989,   1.00000,   0.85824,   0.00000],
        [ 39.97903, 221.66737,  65.53487, 300.42285,   2.00000,   0.80171,   0.00000],
        [494.04330, 224.41110, 511.82700, 280.30139,   3.00000,   0.79048,   0.00000],
        [431.94629, 223.09326, 450.54474, 275.00488,   4.00000,   0.77511,   0.00000],
        [ 93.43863, 232.28745, 115.77039, 303.86487,   5.00000,   0.75890,   0.00000],
        [127.76765, 223.25015, 158.64429, 308.86588,   6.00000,   0.72778,   0.00000]])
tensor([[182.47154, 221.76950, 216.54716, 330.98077,   1.00000,   0.86848,   0.00000],
        [ 39.73378, 220.59914,  64.74668, 300.02478,   2.00000,   0.81102,   0.00000],
        [493.57944, 226.51631, 511.66278, 280.44531,   3.00000,   0.68096,   0.00000],
        [431.88257, 223.42232, 450.45923, 275.16696,   4.00000,   0.73158,   0.00000],

AttributeError: 'NoneType' object has no attribute 'cpu'

In [21]:
trackout.release()

In [20]:
Handler.object_container.keys()

dict_keys([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 9.0])

In [23]:
_=Handler.createDataframe()

,frame_num,track_id,class,color,temperature,spatials,keypoints
0,0.0,1,0,"[6, 2, 0]",70.749798,"[-1.4658338962239688, -0.5302913039263073, 9.1...","[[209.66556, 231.25336, 0.7894298], [209.66556..."
1,0.0,2,0,"[6, 5, 1]",60.338636,"[-8.19005841147257, -0.20330192733994118, 14.2...","[[56.98698, 228.35852, 0.6702892], [55.448788,..."
2,0.0,3,0,"[18, 26, 53]",94.372960,"[4.276339491283203, 0.06310128413258408, 6.117...","[[498.84164, 232.43442, 0.84143305], [499.3874..."
3,0.0,4,0,"[68, 88, 108]",92.652116,"[15.114705920896274, 0.56714587231576, 28.8235...","[[440.4851, 229.02457, 0.7544142], [441.49896,..."
4,0.0,5,0,"[0, 0, 0]",70.993255,"[-4.285660742707637, -0.34184885463823544, 10.0]","[[111.94399, 238.36876, 0.77869993], [111.9439..."
...,...,...,...,...,...,...,...
806,101.0,10,0,"[69, 56, 38]",79.572277,"[-0.44718700291307517, -0.21874815469956552, 1...","[[250.95839, 235.05606, 0.7086398], [250.95839..."
807,101.0,13,0,"[9, 15, 16]",81.422095,"[-3.506243810322791, 0.36014117998764766, 29.1...","[[213.32083, 233.2891, 0.5201303], [211.96153,..."
808,101.0,14,0,"[14, 17, 19]",85.126804,"[6.127758146493221, -0.5781917847670918, 14.70...","[[395.11765, 236.28653, 0.5968559], [395.96835..."
809,101.0,16,0,"[12, 9, 6]",93.142105,"[2.2480723059341874, -0.07755520470547045, 6.8...","[[370.17047, 233.12646, 0.9294093], [371.4358,..."


In [ ]:
# self.properties = {
#             "class": cls,
#             "id": idenity,
#             "color": [color],
#             "temperature": [temperature],
#             "spatials": [spatials],
#             "pose": [pose],
#             "frame": [frame]
#         }

In [5]:
trackout.release()

In [ ]:
csv = ObjList.createDataframe()
csv.to_csv(f"videos/{fileName}/processed/object_list.csv", index=False)

In [ ]:
weights = {"Color": 0.15, "Temperature": 0.15, "Pose": 0.40, "Depth": 0.10, "Bounding Box": 0.20}
x = [1, 1, 1, 1, 1]
x1 = list(weights.values())
print(x, x1)
np.dot(x, x1)

In [ ]:
img = images["RGB"].copy()


for obj in ObjList.ObjectList:
    print(len(obj.bb_history))
    for bb in obj.bb_history:
        color = [int(np.random.rand(1)*255), int(np.random.rand(1)*255), int(np.random.rand(1)*255)] 
        img = cv2.circle(img, bb[:2], 5, color , 5)
        
        cv2.imshow("HAHAHAAHAH", img)
        cv2.waitKey(1)
        sleep(0.5)
    

In [47]:
tuple(np.random.randint([255, 255, 255]))

(192, 23, 180)

In [ ]:
cv2.destroyAllWindows()

In [ ]:
x = 1
print(-x)

In [ ]:
c1 = np.array([0,0,0])
c2 = np.array([2,5,7])
sum( abs(c1 - c2) ) / 765

In [ ]:
x = np.array([1])

type(x) == type(None)

array([224, 117, 161])

In [29]:
timeNow = str(datetime.now())[:-7].replace(" ", "_").replace(":", "-")
print(timeNow)

NameError: name 'datetime' is not defined